### Import

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import shapefile
import re
import pickle
import time
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from tqdm import tqdm
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist

### Load Tract Data

In [2]:
tractdata = pd.read_csv('../data/acs/nyc_tracts/nyc_tracts_census_geo_node.csv')

### Load Chain Data

In [3]:
chaindata = pd.read_csv('../data/tables/chains_with_nodes.csv')

### Load Graph

In [4]:
X = pickle.load(open("../data/graphs/nyc.p","rb"))
O = X['O']
N = X['N']
edges = X['edges']
nodes = X['nodes']
nodenames = X['nodenames']
edgedatabase = X['edgedatabase']
streets = X['streets']
G = X['G']
Nlist = X['Nlist'].tolist()

### Create Tract Index List

In [5]:
tractlist = []
for i in range(tractdata.shape[0]):
    tractdp = tractdata.iloc[i]
    tract_node_name = tractdp.node_name.split(' | ')
    tract_node_name = ' | '.join(tract_node_name[1:])
    index = Nlist.index(tract_node_name)
    tractlist.append(index)
tractdata['node_index'] = tractlist

### Create Chain Index List

In [6]:
chainlist = []
chainnames = []
for i in range(chaindata.shape[0]):
    chaindp = chaindata.iloc[i]
    chain_node_name = chaindp.nodename
    index = Nlist.index(chain_node_name)
    chainlist.append(index)
chaindata['indexer'] = range(chaindata.shape[0])
chaindata['chain_address'] = chaindata['indexer'].astype(str) + ' | '  + chaindata['chain'] + ' | ' + chaindata['address'] + ' | ' + chaindata['zipcode'].astype(str)
chainnames = np.array(chaindata['chain'].values)
chainaddress = np.array(chaindata['chain_address'].values)
chaindata['node_index'] = chainlist

### Get Lengths

In [7]:
def computeLength(path,G):
    
    #Compute Length
    lengths = []
    for i in range(len(path)-1):
        orig = path[i]
        dest = path[i+1]
        weight = G[orig][dest]['weight']
        lengths.append(weight)

    #Return
    return lengths

### Pair Exists in data

In [8]:
def pairExists(source,target,chain,P):

    #Top Level miss
    exists = False
    if (source not in P):
        exists = False

    #Source in top level P
    if (source in P):
        
        #Get Target Items
        target_items = P[source]
        
        #Loop through target items to see if they are keyed on target
        for target_item in target_items:
            key = list(target_item.keys())[0]

            #If target is in P[source] determine whether chain occurs in target item
            if (target == key):
                chains,distance = target_item[key]
                if (chain in chains):
                    exists = True
    
    #Return
    return exists

### Create Mesh

In [9]:
# stepSize = 10
# tractmesh = list(range(0,len(tractlist),stepSize))
# chainmesh = list(range(0,len(chainlist),stepSize))
# tractmesh[-1] = len(tractlist)
# chainmesh[-1] = len(chainlist)
# M = {}
# V = np.zeros((len(tractmesh)-1,len(chainmesh)-1))
# for i in range(len(tractmesh)-1):
#     for j in range(len(chainmesh)-1):
#         xstart = tractmesh[i]
#         xend = tractmesh[i+1]
#         ystart = chainmesh[j]
#         yend = chainmesh[j+1]
#         M[i,j] = (xstart,xend,ystart,yend)

# X = {}
# X['M'] = M
# X['V'] = V
# pickle.dump(X,open('../data/meshes/nyc_mesh.p',"wb"))    

### Store and Update Mesh

In [10]:
def storeAndUpdate(M,V):
    
    #Update and Store
    X = {}
    X['M'] = M
    X['V'] = V
    pickle.dump(X,open('../data/meshes/nyc_mesh.p',"wb"))

### Pick Random Submesh and Return 

In [11]:
def pickSubmesh():

    #Unpack
    X = pickle.load(open('../data/meshes/nyc_mesh.p',"rb"))
    M = X['M']
    V = X['V']

    #Return -1 if V is completely filled
    if (np.sum(V) == (V.shape[0] * V.shape[1])):
        return (-1,-1,-1,-1),-1,-1,M,V
        
    #Pick Random submush
    (posi,posj) = np.where(V < 1.0)
    thisi = posi[np.random.randint(len(posi))]
    thisj = posj[np.random.randint(len(posj))]
    return M[(thisi,thisj)],thisi,thisj,M,V

### Find Paths

In [12]:
def findPaths(xs,xe,ys,ye,tractlist,chainlist,durations,G,D,P,unconnected):
    
    #Find Paths if necessary
    durations = []
    if (xs > -1):

        #Loop
        for i in range(xs,xe):
            starttime = time.time()
            for j in range(ys,ye):
                if (D[i,j] == 0):
                    
                    #Get Source and Target Node Indices
                    source =  tractlist[i]
                    target = chainlist[j]

                    #Try to run shortest path algorithm
                    try:
                        path = nx.shortest_path(G,source=source,target=target)
                        pathlengths = computeLength(path,G)
                    except:
                        unconnected.append({source_index,target_index})
                        path = []
                        pathlengths = []

                    #Find all subaths between tract and chain
                    for p1 in range(len(path)):
                        for p2 in range(p1+1,len(path)):
                            source_index = path[p1]
                            target_index = path[p2]
                            if (source_index in tractlist):
                                if (target_index in chainlist):
                                    source_sub = tractlist.index(source_index)
                                    target_sub = chainlist.index(target_index)
                                    sub_length = np.sum(pathlengths[p1:p2])
                                    sub_path = path[p1:p2]
                                    D[source_sub,target_sub] = sub_length
                                    P[source_sub] = {target_sub:sub_path}

                    #Find all subaths between tract and chain (reverse)
                    path = path[::-1]
                    pathlengths = pathlengths[::-1]
                    for p1 in range(len(path)):
                        for p2 in range(p1+1,len(path)):
                            source_index = path[p1]
                            target_index = path[p2]
                            if (source_index in tractlist):
                                if (target_index in chainlist):
                                    source_sub = tractlist.index(source_index)
                                    target_sub = chainlist.index(target_index)
                                    sub_length = np.sum(pathlengths[p1:p2])
                                    D[source_sub,target_sub] = sub_length
                                    P[source_sub] = {target_sub:sub_path}

            #Stop the clock            
            endtime = time.time()
            durations.append(endtime-starttime)

    #Return
    return D,P,durations,unconnected

### Find Paths with Intermediate Saves

In [ ]:
for i in range(1000):
    
    #Get Submesh
    (xs,xe,ys,ye),posi,posj,M,V = pickSubmesh()

    #Load Intermediate
    if (os.path.isfile('../data/paths/nyc_paths.p')):
        X = pickle.load(open('../data/paths/nyc_paths.p',"rb"))
        D = X['D']
        P = X['P']
        durations = X['durations']
        unconnected = X['unconnected']
    else:
        unconnected = []
        durations = []
        D = np.zeros((len(tractlist),len(chainlist)))
        P = {}

    #Run if matrix is already more or less full
    Dsubset = D[xs:xe,ys:ye]
    if (np.sum(Dsubset>0) > 10):
        print(i,'start run...')
        print(float(np.sum(D>1)) / float(D.shape[0]*D.shape[1]) * 100.0)
        print(float(np.sum(V) /float(V.shape[0]*V.shape[1])) * 100.0)

        #Find Paths
        D,P,durations,unconnected = findPaths(xs,xe,ys,ye,tractlist,chainlist,durations,G,D,P,unconnected)

        #Store Intermediate
        X = {}
        X['D'] = D
        X['P'] = P
        X['durations'] = durations
        X['unconnected'] = unconnected
        pickle.dump(X,open('../data/paths/nyc_paths.p',"wb"))

        #Update Mesh
        V[posi,posj] = 1
        storeAndUpdate(M,V)

0 start run...
15.10445778541646
3.8306588123772767
6 start run...
15.106263838140544
3.831505179768434
7 start run...
15.108576260787263
3.832351547159591
8 start run...
15.109361134167354
3.833197914550748
10 start run...
15.11037387401263
3.8340442819419054
11 start run...
15.112711615155483
3.834890649333063
14 start run...
15.113606202018811
3.8357370167242193
15 start run...
15.115344738753208
3.8365833841153765
18 start run...
15.116644421554648
3.837429751506534
19 start run...
15.117243625963106
3.8382761188976913
20 start run...
15.11975015708017
3.8391224862888484
23 start run...
15.121286145845508
3.8399688536800056
24 start run...
15.122526752155977
3.8408152210711624
25 start run...
15.123716721474178
3.8416615884623195
26 start run...
15.12530334723178
3.8425079558534767
29 start run...
15.12577595915958
3.843354323244634
30 start run...
15.12836688526375
3.8442006906357915
32 start run...
15.12836688526375
3.8442006906357915
33 start run...
15.129750963052297
3.84504705

In [14]:
print(np.sum(D>1))
print(D.shape[0]*D.shape[1])
print(100.0*float(np.sum(D>1))/float(D.shape[0]*D.shape[1]))
#prev1 = 780317
#prev2 = 829720
#prev3 = 875274
#prev4 = 991589
#prev5 = 1015162
#prev6 = 1170880
#prev7 = 1542826
#prev8 = 1789734

1789734
11849045
15.104457785416463


### Create Node space with distance a tract

In [ ]:
data = []
mychain = 'McDonalds'
tractkeys = list(P.keys())
nodeindex1 = tractkeys[0]
for item in P[nodeindex1]:
    nodeindex2,mychainlist,mydistance = nodeUnpacker(item)
    node1 = N[nodeindex1]
    node2 = N[nodeindex2]
    elements1 = node1.split(' | ')
    elements2 = node2.split(' | ')
    lat1 = float(elements1[-3])
    lng1 = float(elements1[-2])
    lat2 = float(elements2[-3])
    lng2 = float(elements2[-2])
    chainindices = np.where(chainlist == nodeindex2)[0]
    for c,chain_address in enumerate(mychainlist):
        elements = chain_address.split(' | ')
        data.append((lat1,lng1,lat2,lng2,chain_address,elements[1],mydistance))
data = pd.DataFrame(data)
data.columns = ['lat1','lng1','lat2','lng2','chain+address','chain','distance']
data = data[data['chain'] == mychain]

### Normalize

In [ ]:
distances = data['distance'].values
data['distance_norm'] = distances / np.max(distances)

### Load Outline

In [ ]:
sf = shapefile.Reader('../data/shapefiles/nyc_outline/nyc_outline.shp')
streetsShapeRecs = sf.shapeRecords()
points = np.array(streetsShapeRecs[0].shape.points)
parts = streetsShapeRecs[0].shape.parts
parts.append(points.shape[0])

### Plot

In [ ]:
B = data[['lat1','lng1']].values
X = data[['lat2','lng2']].values
rgb = np.zeros((X.shape[0],3))
rgb[:,0] = data['distance_norm'].values
rgb[:,1] = 0#data['distance_norm'].values**2
rgb[:,2] = 1-data['distance_norm'].values
offset = 0.02
skip = 1
plt.figure(figsize=(9,9))
for i in range(len(parts)-1):
    spos = parts[i]
    epos = parts[i+1]
    polygon = points[spos:epos,:]
    plt.plot(polygon[:,0],polygon[:,1],c='w',linewidth=1);
plt.scatter(X[:,0],X[:,1],c=rgb[:,:],s=60)
plt.scatter(B[0,0],B[0,1],c='w',s=60)
minval = np.min(np.concatenate((B[:1,:],X),axis=0),axis=0)
maxval = np.max(np.concatenate((B[:1,:],X),axis=0),axis=0)
plt.axis((minval[0]-offset,maxval[0]+offset,minval[1]-offset,maxval[1]+offset));

### Get Found Tracts

In [ ]:
tracts = []
tractkeys = list(P.keys())
for key in tractkeys:
    index = np.where(tractlist == key)[0][0]
    tractdp = tractdata.iloc[index]
    lat = tractdp.lat
    lng = tractdp.lng
    tracts.append((lat,lng,tractdp.geoid))
tracts = pd.DataFrame(tracts)
tracts.columns = ['lat','lng','geoid']

### Get Chains

In [ ]:
chains = []
tractkeys = list(P.keys())
indices = []
for key in tractkeys:
    items = P[key]
    for item in items:
        nodeindex,mychainlist,mydistance = nodeUnpacker(item)
        for mychain in mychainlist:
            elements = mychain.split(' | ')
            index = int(elements[0])
            indices.append(index)
            chaindp = chaindata.iloc[index]
            lat = chaindp.lat
            lng = chaindp.lng
            chains.append((lat,lng,chaindp.chain))
chains = pd.DataFrame(chains)
chains.columns = ['lat','lng','node']
chains = chains.drop_duplicates(keep='first')

### Load Tract Shapefile

In [ ]:
sf = shapefile.Reader('../data/acs/nyc_tracts/nyc_tracts_cleaned.shp')
tractsShapeRecs = sf.shapeRecords()
plist = []
for i in range(len(tractsShapeRecs)):
    tractpoints = np.array(tractsShapeRecs[i].shape.points)
    tractparts = tractsShapeRecs[i].shape.parts
    tractparts.append(tractpoints.shape[0])
    plist.append((tractpoints,tractparts))

### Plot Tracts and Chains

In [ ]:
X = tracts[['lng','lat']].values
C = chains[['lng','lat']].values
rgb = np.zeros((X.shape[0],3))
offset = 0.02
skip = 1
plt.figure(figsize=(9,9))
for m in range(len(plist)):
    tractpoints,tractparts = plist[m]
    for i in range(len(tractparts)-1):
        spos = tractparts[i]
        epos = tractparts[i+1]
        polygon = tractpoints[spos:epos,:]
        plt.plot(polygon[:,0],polygon[:,1],c='w',linewidth=0.5,zorder=1)
plt.scatter(X[:,0],X[:,1],c='r',s=10,zorder=2)
plt.scatter(C[:,0],C[:,1],c='g',s=10,zorder=3)
minval = np.min(X,axis=0)
maxval = np.max(X,axis=0)
plt.axis((minval[0]-offset,maxval[0]+offset,minval[1]-offset,maxval[1]+offset));

### Save P

In [ ]:
X = {}
X['P'] = P
X['D'] = D
pickle.dump(X,open('../data/paths/nyc_tracts_nodes.p',"wb"))